 # Explore AMP-PD data available for the LID project

## Load libs, env variables, utils, and get path to data

In [1]:
# Use the os package to interact with the environment
import os

# Bring in Pandas for Dataframe functionality
import pandas as pd

# numpy for basics
import numpy as np

# To do tome regex ops
import re

# Use StringIO for working with file contents
from io import StringIO

# Enable IPython to display matplotlib graphs
import matplotlib.pyplot as plt
%matplotlib inline

# Enable interaction with the FireCloud API
from firecloud import api as fapi

# Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.core.display import display, HTML

# Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

# BigQuery for querying data
from google.cloud import bigquery

import sys

In [2]:
# pip install some python pkg to do cox proportional hazard analysis
! pip install lifelines
! pip show lifelines

Name: lifelines
Version: 0.27.1
Summary: Survival analysis in Python, including Kaplan Meier, Nelson Aalen and regression
Home-page: https://github.com/CamDavidsonPilon/lifelines
Author: Cameron Davidson-Pilon
Author-email: cam.davidson.pilon@gmail.com
License: MIT
Location: /home/jupyter/.local/lib/python3.7/site-packages
Requires: autograd, autograd-gamma, formulaic, matplotlib, numpy, pandas, scipy
Required-by: 


In [3]:
%%capture 

sys.path.insert(0, '/home/jupyter/.local/lib/python3.7/site-packages')
from lifelines import CoxPHFitter

In [4]:
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
WORKSPACE_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE_NAME = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']


In [5]:
WORKSPACE_ATTRIBUTES = fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME).json().get('workspace',{}).get('attributes',{})

GS_RELEASE_PATH = 'gs://amp-pd-data/releases/2021_v2-5release_0510'
GS_CLINICAL_RELEASE_PATH = f'{GS_RELEASE_PATH}/clinical/'

GS_WGS_RELEASE_PATH = 'gs://amp-pd-genomics/releases/2021_v2-5release_0510/wgs'
GS_WGS_RELEASE_PLINK_PATH = os.path.join(GS_WGS_RELEASE_PATH, 'plink')

BQ_RELEASE_DATASET = 'amp-pd-research.2021_v2-5release_0510'

In [6]:
# Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}')
    !$command

def shell_return(command):
    print(f'Executing: {command}', file=sys.stderr)
    output = !$command
    return '\n'.join(output)


# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))
    
# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)
    
def gcs_read_csv(path, sep=None):
    """Return a DataFrame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)
    
# Get the data from a query
def bq_query(query):
    print(f'Executing: {query}', file=sys.stderr)
    return pd.read_gbq(query, project_id=BILLING_PROJECT_ID, dialect='standard')

# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)
    
# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_bq(description, link_text, bq_dataset, bq_table=None):
    project, dataset = bq_dataset.split('.', 1)
    if bq_table:
        page_params = {'page': 'table', 'p': project, 'd': dataset, 't': bq_table}
    else:
        page_params = {'page': 'dataset', 'p': project, 'd': dataset}
    
    url = '{}?{}'.format(
        'https://console.cloud.google.com/bigquery',
        urllib.parse.urlencode(page_params))

    display_html_link(description, link_text, url)    

# Utility routine for printing a query before executing it
def bq_query(query):
    """Return the contents of a query against BigQuery"""
    return pd.read_gbq(
        query,
        project_id=BILLING_PROJECT_ID,
        dialect='standard')

In [7]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GS_RELEASE_PATH}')
#shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GS_CLINICAL_RELEASE_PATH}')

Executing: gsutil -u terra-1cae21c2 ls gs://amp-pd-data/releases/2021_v2-5release_0510
gs://amp-pd-data/releases/2021_v2-5release_0510/amp_pd_case_control.csv
gs://amp-pd-data/releases/2021_v2-5release_0510/amp_pd_participant_wgs_duplicates.csv
gs://amp-pd-data/releases/2021_v2-5release_0510/amp_pd_participants.csv
gs://amp-pd-data/releases/2021_v2-5release_0510/rna_sample_inventory.csv
gs://amp-pd-data/releases/2021_v2-5release_0510/wgs_gatk_joint_genotyping_samples.csv
gs://amp-pd-data/releases/2021_v2-5release_0510/wgs_sample_inventory.csv
gs://amp-pd-data/releases/2021_v2-5release_0510/clinical/


In [8]:
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} ls {WORKSPACE_BUCKET}')

Executing: gsutil -mu terra-1cae21c2 ls gs://fc-509fb8f7-9df1-43ec-805d-e976c08ce034
gs://fc-509fb8f7-9df1-43ec-805d-e976c08ce034/clinical_data/
gs://fc-509fb8f7-9df1-43ec-805d-e976c08ce034/covar_files/
gs://fc-509fb8f7-9df1-43ec-805d-e976c08ce034/notebooks/


## Get and explore AMP-PD PD data

In [9]:
pd_case_control_df = gcs_read_csv(os.path.join(GS_RELEASE_PATH, 'amp_pd_case_control.csv'))
pd_case_control_df.info()
#pd_case_control_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10772 entries, 0 to 10771
Data columns (total 5 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   participant_id                  10772 non-null  object
 1   diagnosis_at_baseline           10772 non-null  object
 2   diagnosis_latest                10772 non-null  object
 3   case_control_other_at_baseline  10772 non-null  object
 4   case_control_other_latest       10772 non-null  object
dtypes: object(5)
memory usage: 420.9+ KB


In [10]:
# Subset and keep only latest diagnosis 
pd_case_control_df_latest = pd_case_control_df[['participant_id', 'diagnosis_latest', 'case_control_other_latest']]
pd_case_control_df_latest.columns = ['ID', 'LATEST_DX', 'CASE_CONTROL']

#print(pd_case_control_df_noPPMI_latest.value_counts())
print(pd_case_control_df_latest['LATEST_DX'].value_counts())

No PD Nor Other Neurological Disorder        4312
LBD                                          2614
Parkinson's Disease                          2392
Idiopathic PD                                1135
Multiple System Atrophy                        71
Progressive Supranuclear Palsy                 61
Essential Tremor                               47
Prodromal non-motor PD                         36
Prodromal motor PD                             36
Other Neurological Disorder(s)                 30
Dementia With Lewy Bodies                      14
Corticobasal Degeneration                       8
Neuroleptic-Induced Parkinsonism                3
Parkinsonism                                    3
Possible Alzheimer's Disease                    3
Vascular Parkinsonism                           2
Fahr's Syndrome                                 1
Alzheimer's Disease                             1
Psychogenic Illness                             1
Juvenile Autosomal Recessive Parkinsonism       1


In [11]:
# Grab the enrollment info so we know how each patient was recruited 
enrollment_df = gcs_read_csv(os.path.join(GS_CLINICAL_RELEASE_PATH, 'Enrollment.csv'))
enrollment_subset_df = enrollment_df[['participant_id', 'study_arm']].copy()
enrollment_subset_df.columns = ['ID', 'ENROLL_STUDY_ARM']

print(enrollment_subset_df['ENROLL_STUDY_ARM'].value_counts(dropna=False))

Healthy Control                3377
PD                             2987
LBD                            2521
Genetic Cohort Unaffected       403
Genetic Cohort PD               272
Genetic Registry Unaffected     245
Genetic Registry PD             196
Disease Control                 159
SWEDD                            77
Prodromal                        64
NaN                               3
Unknown                           1
Name: ENROLL_STUDY_ARM, dtype: int64


In [12]:
demo_enrollment_df = pd_case_control_df_latest.merge(enrollment_subset_df, on="ID", how="left")
demo_enrollment_df.info()
#demo_enrollment_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11054 entries, 0 to 11053
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                11054 non-null  object
 1   LATEST_DX         11054 non-null  object
 2   CASE_CONTROL      11054 non-null  object
 3   ENROLL_STUDY_ARM  10302 non-null  object
dtypes: object(4)
memory usage: 431.8+ KB


In [13]:
# Coding for PD_PHENO in AMP data
    # 1=Healthy control with no neurological disorders
    # 2=Positive PD case not in a genetic enrichment study, prodromal, or a SWEDD
conditions = [
    (demo_enrollment_df['ENROLL_STUDY_ARM'] == "Healthy Control") & (demo_enrollment_df['LATEST_DX'] == "No PD Nor Other Neurological Disorder"),
    (demo_enrollment_df['ENROLL_STUDY_ARM'] == "Healthy Control") & (demo_enrollment_df['LATEST_DX'] == "Idiopathic PD"),
    (demo_enrollment_df['ENROLL_STUDY_ARM'] == "PD") & (demo_enrollment_df['LATEST_DX'] == "Parkinson's Disease"),
    (demo_enrollment_df['ENROLL_STUDY_ARM'] == "PD") & (demo_enrollment_df['LATEST_DX'] == "Idiopathic PD")
]
choices = [1,3,2,2]

demo_enrollment_df['PD_PHENO'] = np.select(conditions, choices, default=-9)
print(demo_enrollment_df['PD_PHENO'].value_counts(dropna=False))
#demo_enrollment_df.loc[demo_enrollment_df['PD_PHENO'] == 3]

-9    4733
 1    3358
 2    2959
 3       4
Name: PD_PHENO, dtype: int64


,ID,LATEST_DX,CASE_CONTROL,ENROLL_STUDY_ARM,PD_PHENO
2989,PP-3160,Idiopathic PD,Case,Healthy Control,3
3011,PP-3191,Idiopathic PD,Case,Healthy Control,3
3064,PP-3310,Idiopathic PD,Case,Healthy Control,3
3158,PP-3478,Idiopathic PD,Case,Healthy Control,3


In [14]:
## Add study as a column 
# Add cohort column (Numpy's NAN for anything missing - though nothing should be missing)
demo_enrollment_df['COHORT'] = np.where(demo_enrollment_df.ID.str.contains("LB-"), "LBD",
                               np.where(demo_enrollment_df.ID.str.contains("PD-"), "PDBP",
                               np.where(demo_enrollment_df.ID.str.contains("PP-"), "PPMI",
                               np.where(demo_enrollment_df.ID.str.contains("HB-"), "HBS",
                               np.where(demo_enrollment_df.ID.str.contains("LC-"), "LCC",
                               np.where(demo_enrollment_df.ID.str.contains("BF-"), "BIOFIND",
                               np.where(demo_enrollment_df.ID.str.contains("SU-"), "SURE-PD3",
                               np.where(demo_enrollment_df.ID.str.contains("SY-"), "STEADY-PD3", np.nan))))))))

print(demo_enrollment_df['COHORT'].value_counts(dropna=True))
demo_enrollment_df.info()

LBD           4586
PPMI          1945
PDBP          1606
HBS           1189
LCC            638
STEADY-PD3     616
SURE-PD3       261
BIOFIND        213
Name: COHORT, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 11054 entries, 0 to 11053
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                11054 non-null  object
 1   LATEST_DX         11054 non-null  object
 2   CASE_CONTROL      11054 non-null  object
 3   ENROLL_STUDY_ARM  10302 non-null  object
 4   PD_PHENO          11054 non-null  int64 
 5   COHORT            11054 non-null  object
dtypes: int64(1), object(5)
memory usage: 604.5+ KB


In [15]:
demo_enrollment_df_PD = demo_enrollment_df[demo_enrollment_df.PD_PHENO.eq(2)]
len(demo_enrollment_df_PD.ID.unique())
demo_enrollment_df_PD.info()
#demo_enrollment_df_PD.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2959 entries, 2802 to 6609
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                2959 non-null   object
 1   LATEST_DX         2959 non-null   object
 2   CASE_CONTROL      2959 non-null   object
 3   ENROLL_STUDY_ARM  2959 non-null   object
 4   PD_PHENO          2959 non-null   int64 
 5   COHORT            2959 non-null   object
dtypes: int64(1), object(5)
memory usage: 161.8+ KB


## Load and explore MDSUPDRSIV data

In [16]:
MDSUPDRSIV_dict = gcs_read_csv(os.path.join(GS_CLINICAL_RELEASE_PATH, 'MDS_UPDRS_Part_IV_dictionary.csv'))
MDSUPDRSIV_dict.info()
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    display(MDSUPDRSIV_dict)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   TableName      17 non-null     object
 1   ColumnName     17 non-null     object
 2   Title          17 non-null     object
 3   Description    17 non-null     object
 4   Required       17 non-null     bool  
 5   DataType       17 non-null     object
 6   DataTypeRange  7 non-null      object
 7   UniqueValues   6 non-null      object
dtypes: bool(1), object(7)
memory usage: 1.1+ KB


### Potential data that can be used in LID study

In [17]:
MDSUPDRSIV = gcs_read_csv(os.path.join(GS_CLINICAL_RELEASE_PATH, 'MDS_UPDRS_Part_IV.csv'))
MDSUPDRSIV = MDSUPDRSIV[["participant_id", "GUID", "visit_name", "visit_month", "code_upd2401_time_spent_with_dyskinesias"]]
MDSUPDRSIV.columns = ['ID', 'GUID', 'visit_name', 'visit_month', 'MDS_UPDRS_IV_1']

## Add study as a column 
# Add cohort column (Numpy's NAN for anything missing - though nothing should be missing)
MDSUPDRSIV['COHORT'] = np.where(MDSUPDRSIV.ID.str.contains("LB-"), "LBD",
                               np.where(MDSUPDRSIV.ID.str.contains("PD-"), "PDBP",
                               np.where(MDSUPDRSIV.ID.str.contains("PP-"), "PPMI",
                               np.where(MDSUPDRSIV.ID.str.contains("HB-"), "HBS",
                               np.where(MDSUPDRSIV.ID.str.contains("LC-"), "LCC",
                               np.where(MDSUPDRSIV.ID.str.contains("BF-"), "BIOFIND",
                               np.where(MDSUPDRSIV.ID.str.contains("SU-"), "SURE-PD3",
                               np.where(MDSUPDRSIV.ID.str.contains("SY-"), "STEADY-PD3", np.nan))))))))

print(MDSUPDRSIV[['COHORT', 'visit_month']].value_counts(dropna=True).sort_index())
MDSUPDRSIV.info()




COHORT      visit_month
BIOFIND      0.0            118
             0.5             15
HBS          0.0            639
PDBP         0.0           1400
             6.0            591
             12.0           560
             18.0           529
             24.0           493
             30.0           431
             36.0           395
             42.0           159
             48.0           120
             54.0            11
             60.0            36
PPMI        -1.0             45
             0.0            374
             3.0             25
             6.0            241
             9.0            156
             12.0           475
             18.0           484
             24.0           570
             30.0           501
             36.0           489
             42.0           460
             48.0           446
             54.0           379
             60.0           377
             72.0           272
             84.0           237
             96.

    **CONCLUSSIONS**  
PDBP and SURE-PD3 seems good candidates to be used in the LID analysis

In [18]:
demo_enrollment_df_PD_filtered = demo_enrollment_df_PD[demo_enrollment_df_PD['ID'].str.contains("PD-|SU-")]
demo_enrollment_df_PD_filtered = demo_enrollment_df_PD_filtered.drop('COHORT', axis=1)
demo_enrollment_df_PD_filtered

# We inner join the demo_PD file. Then, we come up with all PD patients from 
MDSUPDRSIV_DEMO = demo_enrollment_df_PD_filtered.drop_duplicates().merge(MDSUPDRSIV, 
                                                                         left_on=['ID'], right_on=['ID'],
                                                                         how='inner')

MDSUPDRSIV_DEMO.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4515 entries, 0 to 4514
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                4515 non-null   object 
 1   LATEST_DX         4515 non-null   object 
 2   CASE_CONTROL      4515 non-null   object 
 3   ENROLL_STUDY_ARM  4515 non-null   object 
 4   PD_PHENO          4515 non-null   int64  
 5   GUID              4515 non-null   object 
 6   visit_name        4515 non-null   object 
 7   visit_month       4515 non-null   float64
 8   MDS_UPDRS_IV_1    3821 non-null   float64
 9   COHORT            4515 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 388.0+ KB


### Explore NAs in the outcome. Look at NAs stratified by cohort

In [19]:
MDSUPDRSIV_DEMO.MDS_UPDRS_IV_1.isnull().groupby([MDSUPDRSIV_DEMO['COHORT'], MDSUPDRSIV_DEMO['visit_month']]).sum().astype(int)

COHORT    visit_month
PDBP      0.0              0
          6.0              0
          12.0             0
          18.0             0
          24.0             0
          30.0             0
          36.0             0
          42.0             0
          48.0             0
          54.0             0
          60.0             0
SURE-PD3  0.0            258
          6.0            189
          12.0           109
          18.0            52
          24.0            43
          30.0            43
Name: MDS_UPDRS_IV_1, dtype: int64

In [20]:
#Look at the number of ppl meeting the outcome at each visit
MDSUPDRSIV_DEMO.MDS_UPDRS_IV_1.ge(2).groupby([MDSUPDRSIV_DEMO['COHORT'], MDSUPDRSIV_DEMO['visit_month']]).sum().astype(int)


COHORT    visit_month
PDBP      0.0            47
          6.0            30
          12.0           19
          18.0           23
          24.0           29
          30.0           15
          36.0           26
          42.0           15
          48.0            5
          54.0            1
          60.0            4
SURE-PD3  0.0             0
          6.0             0
          12.0            0
          18.0            3
          24.0            2
          30.0            0
Name: MDS_UPDRS_IV_1, dtype: int64

### Conclussion

**CONCLUSSION**  
I think the NA pattern we see in SURE-PD3 is because MDS-UPDRS IV assessment is not taken until L-dopa treatment starts


**Make decision regarding using SURE-PD3 data or not**

DECISION: There are almost no people developing dyskinesias in SURE-PD. I will exclude this, and work only with PDBP

## PDBP QC

In [21]:
MDSUPDRSIV_DEMO_PDBPD = MDSUPDRSIV[MDSUPDRSIV['ID'].str.contains("PD-")]
MDSUPDRSIV_DEMO_PDBPD.head()

MDSUPDRSIV_DEMO_PDBPD.ID.nunique()


1404

### Remove ppl that have non PD diagnosis

In [22]:
# We inner join the demo_PD file. Then, we come up with all PD patients from 
MDSUPDRSIV_DEMO_PDBPD = demo_enrollment_df_PD_filtered.drop_duplicates().merge(MDSUPDRSIV_DEMO_PDBPD, 
                                                                          left_on=['ID'], right_on=['ID'],
                                                                          how='inner')
MDSUPDRSIV_DEMO_PDBPD.head()
MDSUPDRSIV_DEMO_PDBPD.ID.nunique()

827

### Remove people that have never had levodopa during the length of the study

In [23]:
medical_hist = gcs_read_csv(os.path.join(GS_CLINICAL_RELEASE_PATH, 'PD_Medical_History.csv'))
medical_hist.info()
#medical_hist.head()
print("Levodopa info values", medical_hist.on_levodopa.unique())


medical_hist_PDBP = medical_hist[medical_hist["participant_id"].isin(MDSUPDRSIV_DEMO_PDBPD.ID.unique())]
medical_hist_PDBP.shape

#Checking we have the expacted number of PD patients

print("\nNumber of PD patients with medical history" ,len(medical_hist_PDBP.participant_id.unique()))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28727 entries, 0 to 28726
Data columns (total 20 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   participant_id                                  28727 non-null  object 
 1   GUID                                            9166 non-null   object 
 2   visit_name                                      28727 non-null  object 
 3   visit_month                                     27724 non-null  float64
 4   diagnosis                                       26093 non-null  object 
 5   initial_diagnosis                               252 non-null    object 
 6   most_recent_diagnosis                           249 non-null    object 
 7   change_in_diagnosis                             826 non-null    object 
 8   change_in_diagnosis_months_after_baseline       14 non-null     float64
 9   surgery_for_parkinson_disease          

In [24]:
medical_hist_PDBP_LDOPA = medical_hist_PDBP.loc[(medical_hist_PDBP['on_levodopa']=='Yes')]
print("\nNumber of PD patients that have taken Leovodpa at least once" ,len(medical_hist_PDBP_LDOPA.participant_id.unique()))
medical_hist_PDBP_LDOPA.head()

# We save these patients IDs
PD_IDs_QC = set(medical_hist_PDBP_LDOPA.participant_id.unique())


Number of PD patients that have taken Leovodpa at least once 717


In [25]:
medical_hist_PDBP_LDOPA.on_levodopa.unique()

array(['Yes'], dtype=object)

### Identify ppl with disease duration from BL to diagnosis > 10 years

Rationale of doing this is that the period of uncertainty is too wide.  
These people could have developed LID any time within those 6 years  
Then, treatment amendment according to LID could prevent us from detecting potential LID devevelopers

In [26]:
#medical_hist_PDBP_LDOPA_BL = medical_hist_PDBP_LDOPA[
#    medical_hist_PDBP_LDOPA[["participant_id", "age_at_diagnosis", "visit_name"]]["visit_name"] == "M0"]
#medical_hist_PDBP_LDOPA_BL
AAD = medical_hist_PDBP_LDOPA[["participant_id", "visit_name", "age_at_diagnosis"]][medical_hist_PDBP_LDOPA['age_at_diagnosis'].notnull()]
# Get all non na entries a
AAD_QC = AAD.drop_duplicates(subset = "participant_id")


# Filter the AAD_QC by the total number of samples with 

# We are loosing some samples that do not have AAD
print("Number of samples with AAD data:",AAD_QC.shape[0])
#print(AAD_QC.head())



Number of samples with AAD data: 696


#### Keep LID developers missing AAD

In [27]:
AAD = medical_hist_PDBP_LDOPA[["participant_id", "visit_name", "age_at_diagnosis"]][medical_hist_PDBP_LDOPA['age_at_diagnosis'].notnull()]
# Get all non na entries a
AAD_QC = AAD.drop_duplicates(subset = "participant_id")

# We are loosing some samples that do not have AAD
print("Number of samples with AAD data:",AAD_QC.shape[0])

# Keep people missing AAD and developing LID. 
# If they are LID developers I will consier includind them
AAD_MISSING = set(PD_IDs_QC).difference(set(list(AAD_QC.participant_id)))

# Check if AAD_MISSING ppl develop dyskinesias
MDSUPDRSIV_AAD_MISSING = MDSUPDRSIV_DEMO_PDBPD[MDSUPDRSIV_DEMO_PDBPD["ID"].isin(AAD_MISSING)]
MDSUPDRSIV_AAD_MISSING.MDS_UPDRS_IV_1.ge(2).groupby([MDSUPDRSIV_AAD_MISSING['COHORT'], MDSUPDRSIV_AAD_MISSING['visit_month']]).sum().astype(int)

# Get the IIDs of the three people that develop LID and that are missing AAD.
LID_AADMISSING = set(MDSUPDRSIV_AAD_MISSING[MDSUPDRSIV_AAD_MISSING.MDS_UPDRS_IV_1.ge(2)]["ID"])


# We exclude the LID_AADMISSING ppl from the AAD_MISSING set
AAD_MISSING_EXCLUDE = AAD_MISSING - LID_AADMISSING
[PD_IDs_QC.remove(i) for i in AAD_MISSING_EXCLUDE]
print("\nNumber of PD patients that have AAD including three imputed samples" ,len(PD_IDs_QC))

# Then, we include these 3 ppl in AAOD_QC
#AAD = medical_hist_PDBP_LDOPA[["participant_id", "visit_name", "age_at_diagnosis"]][medical_hist_PDBP_LDOPA["participant_id"].isin(PD_IDs_QC)]
#AAD_QC = AAD.drop_duplicates(subset = "participant_id")
#print("Number of samples with AAD data including 3 LID developers:",AAD_QC.shape[0])

Number of samples with AAD data: 696

Number of PD patients that have AAD including three imputed samples 700


In [28]:
len(PD_IDs_QC)

700

In [29]:
# get age at BL from demographics df and AAD from medical history. Then merge
demographics = gcs_read_csv(os.path.join(GS_CLINICAL_RELEASE_PATH, 'Demographics.csv'))
demographics.info()

demo_AAD = pd.merge(demographics[demographics["visit_name"] == "M0"],
                    AAD_QC[["participant_id", "age_at_diagnosis"]],
                    on = "participant_id",
                    how = "right")

# Calculate disease duration from diagnosis
demo_AAD["DIS_DUR"] = demo_AAD.age_at_baseline - demo_AAD.age_at_diagnosis

demo_AAD_DISDURLONG = list(demo_AAD[demo_AAD.DIS_DUR.ge(9)].participant_id)
len(demo_AAD_DISDURLONG)

# There are 70 ppl with long disease duration.
# We will perform an initial run with and without including these

MDSUPDRSIV_DEMO_PDBPD = MDSUPDRSIV_DEMO_PDBPD[~MDSUPDRSIV_DEMO_PDBPD["ID"].isin(demo_AAD_DISDURLONG)]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11226 entries, 0 to 11225
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   participant_id         11226 non-null  object
 1   GUID                   3603 non-null   object
 2   visit_name             11226 non-null  object
 3   visit_month            11226 non-null  int64 
 4   age_at_baseline        11226 non-null  int64 
 5   sex                    11226 non-null  object
 6   ethnicity              6621 non-null   object
 7   race                   11207 non-null  object
 8   education_level_years  6640 non-null   object
dtypes: int64(2), object(7)
memory usage: 789.5+ KB


In [30]:
MDSUPDRSIV_DEMO_PDBPD.ID.unique().size

619

### Remove patients that only have CS data

In [31]:
# Look at the number of entries at each visit
#MDSUPDRSIV_DEMO_PDBPD.groupby(['visit_name'])['ID'].count()

ID_COUNTS = MDSUPDRSIV_DEMO_PDBPD.groupby(['ID'])['visit_name'].size().reset_index(name='counts')

ID_EXCLUDE = list(ID_COUNTS[ID_COUNTS.counts.eq(1)].ID)

MDSUPDRSIV_DEMO_PDBPD_LG=MDSUPDRSIV_DEMO_PDBPD[~MDSUPDRSIV_DEMO_PDBPD["ID"].isin(ID_EXCLUDE)]

MDSUPDRSIV_DEMO_PDBPD_LG.shape

print("\n\n",len(MDSUPDRSIV_DEMO_PDBPD_LG.ID.unique()),"Patients with longitudinal data available")
#MDSUPDRSIV_DEMO_PDBPD_LG.head()



 345 Patients with longitudinal data available


In [32]:
MDSUPDRSIV_DEMO_PDBPD_LG.groupby(['visit_name'])['visit_name'].count()

visit_name
M0     345
M12    314
M18    289
M24    277
M30    248
M36    215
M42    101
M48     72
M54      5
M6     334
M60     17
Name: visit_name, dtype: int64

### Remove left censored people

In 1.3.2 we could see how there is a total of 45 PD patients that already had dyskinesias at BL.
Remove these.


In [33]:
# Get baseline data and retrieve the IDs of ppl with MDSUDPRS_IV ge(2)

CENSORED_PATIENTS = list(MDSUPDRSIV_DEMO_PDBPD_LG.loc[(MDSUPDRSIV_DEMO_PDBPD_LG['visit_name']=="M0") & 
                             (MDSUPDRSIV_DEMO_PDBPD_LG['MDS_UPDRS_IV_1']>=2)].ID)

MDSUPDRSIV_DEMO_PDBPD_LG_FILT=MDSUPDRSIV_DEMO_PDBPD_LG[~MDSUPDRSIV_DEMO_PDBPD_LG["ID"].isin(CENSORED_PATIENTS)]

MDSUPDRSIV_DEMO_PDBPD_LG_FILT.shape
print("\n\n",len(MDSUPDRSIV_DEMO_PDBPD_LG_FILT.ID.unique()),"No Left censored patients")



 331 No Left censored patients


### Time to LID outcomes and getting all covariates

#### Get COVARIATES

In [45]:
# We will use the list of PDBP patients saved in 1.4.3 section
# Then, get demographics
PD_IDs_QC = list(MDSUPDRSIV_DEMO_PDBPD_LG_FILT.ID.unique())
PDBP_QC = pd.merge(pd.DataFrame(PD_IDs_QC, columns =['ID']),
                   demographics[["participant_id", "age_at_baseline", "sex"]],
                   left_on = "ID", right_on = "participant_id",
                   how = "left")

# Getting SEX
PDBP_QC['sex'] = np.where(PDBP_QC['sex'] == "Female", 2, 1)


# Get the AAD info
PDBP_QC = pd.merge(PDBP_QC,
                   AAD_QC.drop("visit_name", axis = 1),
                   on = "participant_id",
                   how = "left")
# Check there are 3 ppl missing AAD
#print("Number NAs", PDBP_QC['age_at_diagnosis'].isna().sum())
# Impute the AAD for those three ppl from the AAB
print("Check the total of ppl with AAB NAs: ", PDBP_QC.age_at_baseline.isna().sum())
PDBP_QC['AAD_MISSING'] = PDBP_QC['age_at_diagnosis'].isna().astype(int)
PDBP_QC["AAD_IMPUTED"] = PDBP_QC.age_at_diagnosis.mask(PDBP_QC.age_at_diagnosis.isna(),
                                                      PDBP_QC.age_at_baseline - 
                                                       (PDBP_QC.age_at_baseline - PDBP_QC.age_at_diagnosis.mean()))

# Get a tag to identify ppl with long disease duration

# Recalculate DISEASE DURATION after including 3 LID ppl with AAD imputed
# Calculate disease duration from diagnosis
PDBP_QC["DIS_DUR"] = PDBP_QC.age_at_baseline - PDBP_QC.AAD_IMPUTED
DISDURLONG = list(PDBP_QC[PDBP_QC.DIS_DUR.ge(9)].participant_id)
print("PPL WITH LONG DISEASE DURATION: ", len(DISDURLONG))
PDBP_QC.head()
# Add tag for the 70 ppl with long disease duration. I will exclude them on sensitivity analyses
PDBP_QC['LONG_DISDUR'] = PDBP_QC['participant_id'].isin(DISDURLONG).astype(int)


PDBP_QC["DIS_DUR"] =  PDBP_QC.age_at_baseline - PDBP_QC.AAD_IMPUTED
# Check entried with a negative disease duration
PDBP_QC[PDBP_QC.DIS_DUR < 0]
# Exclude patient with DIS_DUR < -10 Years. Possible error in the recoding for that entry
PDBP_QC = PDBP_QC[~PDBP_QC.ID.str.contains('PD-PDAW438DAN')]
# In addition, set the DISEASE DURATION TO 0 for PPL with a disease duration of -1 or -2
PDBP_QC["DIS_DUR"] = [0 if dur < 0 else dur for dur in PDBP_QC.DIS_DUR]
PDBP_QC.info()
print("\nChecking disease duration is okey \n")
print(PDBP_QC[PDBP_QC.DIS_DUR < 0])






# Add the MDS-UPDRSIIItotal score at BL
#demographics = gcs_read_csv(os.path.join(GS_CLINICAL_RELEASE_PATH, 'MDS_UPDRS_Part_III.csv'))
MDSUPDRSIII = gcs_read_csv(os.path.join(GS_CLINICAL_RELEASE_PATH, 'MDS_UPDRS_Part_III.csv'))
MDSUPDRSIII_BL = MDSUPDRSIII[MDSUPDRSIII.visit_name == "M0"]
PDBP_QC = pd.merge(PDBP_QC,
                   MDSUPDRSIII_BL[["participant_id", "mds_updrs_part_iii_summary_score"]],
                   on = "participant_id",
                   how = "left")
print("Check the number of MDSPUPDRSIII_BL NAs: ", PDBP_QC['mds_updrs_part_iii_summary_score'].isna().sum())

Check the total of ppl with AAB NAs:  0
PPL WITH LONG DISEASE DURATION:  4
<class 'pandas.core.frame.DataFrame'>
Int64Index: 330 entries, 0 to 330
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                330 non-null    object 
 1   participant_id    330 non-null    object 
 2   age_at_baseline   330 non-null    int64  
 3   sex               330 non-null    int64  
 4   age_at_diagnosis  298 non-null    float64
 5   AAD_MISSING       330 non-null    int64  
 6   AAD_IMPUTED       330 non-null    float64
 7   DIS_DUR           330 non-null    float64
 8   LONG_DISDUR       330 non-null    int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 25.8+ KB

Checking disease duration is okey 

Empty DataFrame
Columns: [ID, participant_id, age_at_baseline, sex, age_at_diagnosis, AAD_MISSING, AAD_IMPUTED, DIS_DUR, LONG_DISDUR]
Index: []
Check the number of MDSPUPDRSIII_BL NAs:  0


#### Get OUTCOME

In [46]:
# GETTING THE TIME EVENT OUTCOME
MDSUPDRSIV_OUTCOMES = MDSUPDRSIV_DEMO_PDBPD_LG_FILT[MDSUPDRSIV_DEMO_PDBPD_LG_FILT["ID"].isin(PD_IDs_QC)]


# Get all entries with MDS-UPDRS-IV-1 >= 2
# Group by ID
# Get the first entry (ie ming visit_month value)
MDSUPDRSIV_LID = MDSUPDRSIV_OUTCOMES[MDSUPDRSIV_OUTCOMES.MDS_UPDRS_IV_1.ge(2)]
idx = MDSUPDRSIV_LID.groupby(['ID'])['visit_month'].transform(min) == MDSUPDRSIV_LID['visit_month']
MDSUPDRSIV_LID = MDSUPDRSIV_LID[idx]
# To define the event midpoint, we just substract the visit months ( assessments were performed eveyr 3 months)
MDSUPDRSIV_LID["TIMEMIDPOINT_YEARS"] = (MDSUPDRSIV_LID.visit_month - 3 ) / 12
MDSUPDRSIV_LID["EVENT_DYSK"] = 1

# Get another subset filtering out previous samples
# Group by ID
# Get the last entry (ie ming visit_month value)
MDSUPDRSIV_NOLID = MDSUPDRSIV_OUTCOMES[~MDSUPDRSIV_OUTCOMES.ID.isin(MDSUPDRSIV_LID.ID)]


idx = MDSUPDRSIV_NOLID.groupby(['ID'])['visit_month'].transform(max) == MDSUPDRSIV_NOLID['visit_month']
MDSUPDRSIV_NOLID = MDSUPDRSIV_NOLID[idx]
# To define the event midpoint, we just substract the visit months ( assessments were performed eveyr 3 months)
MDSUPDRSIV_NOLID["TIMEMIDPOINT_YEARS"] = [month / 12 if month == 60 else (month + 3) / 12  
                                               for month in MDSUPDRSIV_NOLID.visit_month]

MDSUPDRSIV_NOLID["EVENT_DYSK"] = 0
print("\nNumber of patients with LID",len(MDSUPDRSIV_LID.ID.unique()))
print("\nNumber of patients without LID",len(MDSUPDRSIV_NOLID.ID.unique()))

MDSUPDRSIV_TIMEPOINT = pd.concat([MDSUPDRSIV_LID[["ID","visit_month", "TIMEMIDPOINT_YEARS", "EVENT_DYSK"]], 
                                  MDSUPDRSIV_NOLID[["ID", "visit_month", "TIMEMIDPOINT_YEARS", "EVENT_DYSK"]]])
MDSUPDRSIV_TIMEPOINT.info()

#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    tmp = MDSUPDRSIV_LID.sort_values(["ID", "visit_month"])
#    display(tmp)
#    display(MDSUPDRSIV_LID)


Number of patients with LID 44

Number of patients without LID 287
<class 'pandas.core.frame.DataFrame'>
Int64Index: 331 entries, 6 to 3176
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  331 non-null    object 
 1   visit_month         331 non-null    float64
 2   TIMEMIDPOINT_YEARS  331 non-null    float64
 3   EVENT_DYSK          331 non-null    int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 12.9+ KB


#### Merge the outcome with covariates

In [47]:
PDBP_QC = pd.merge(PDBP_QC,
                   MDSUPDRSIV_TIMEPOINT,
                   on = "ID",
                   how = "left")


PDBP_QC = PDBP_QC.rename(columns={"mds_updrs_part_iii_summary_score" : "MDSUPDRSIII_BL", "sex" : "SEX"}, errors="raise")

PDBP_QC.drop(["participant_id"], axis=1, inplace=True)
PDBP_QC.info()
#PDBP_QC.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 330 entries, 0 to 329
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  330 non-null    object 
 1   age_at_baseline     330 non-null    int64  
 2   SEX                 330 non-null    int64  
 3   age_at_diagnosis    298 non-null    float64
 4   AAD_MISSING         330 non-null    int64  
 5   AAD_IMPUTED         330 non-null    float64
 6   DIS_DUR             330 non-null    float64
 7   LONG_DISDUR         330 non-null    int64  
 8   MDSUPDRSIII_BL      330 non-null    float64
 9   visit_month         330 non-null    float64
 10  TIMEMIDPOINT_YEARS  330 non-null    float64
 11  EVENT_DYSK          330 non-null    int64  
dtypes: float64(6), int64(5), object(1)
memory usage: 33.5+ KB


#### Get the Ldopa data at the time the event is met
We are going to load the Levodopa only LEDD data Tarek generated some time ago and we will append it to the generated DF

In [37]:
shell_do(f'gsutil ls {WORKSPACE_BUCKET}')
ledd_levo = gcs_read_csv(os.path.join(WORKSPACE_BUCKET, 'clinical_data/PDBP_derived_ledd.csv'))
ledd_levo["ID"] = ("PD-" + ledd_levo.SGUID.map(lambda x: re.sub(r"^\d+\.", "", x))).astype(str)
#ledd_levo

Executing: gsutil ls gs://fc-509fb8f7-9df1-43ec-805d-e976c08ce034
gs://fc-509fb8f7-9df1-43ec-805d-e976c08ce034/clinical_data/
gs://fc-509fb8f7-9df1-43ec-805d-e976c08ce034/covar_files/
gs://fc-509fb8f7-9df1-43ec-805d-e976c08ce034/notebooks/


In [38]:
ledd_levo_bl = ledd_levo[ledd_levo.VisitTypPDBP == "Baseline"]
ledd_levo_bl_qc = ledd_levo_bl[ledd_levo_bl["ID"].isin(PDBP_QC.ID.unique())]
ledd_levo_bl_qc
ledd_levo_bl_qc.describe(include=np.number) 

,LEDD,DailyLevo
count,316.000000,235.000000
mean,527.378165,414.057447
std,307.178024,207.456781
min,1.500000,1.500000
25%,300.000000,300.000000
50%,460.000000,300.000000
75%,670.000000,500.000000
max,1750.000000,1600.000000


In [39]:
shell_do(f'gsutil ls {WORKSPACE_BUCKET}')
ledd_levo = gcs_read_csv(os.path.join(WORKSPACE_BUCKET, 'clinical_data/PDBP_derived_ledd.csv'))
ledd_levo["ID"] = ("PD-" + ledd_levo.SGUID.map(lambda x: re.sub(r"^\d+\.", "", x))).astype(str)

#df['col'] = 'str' + df['col'].astype(str)

ledd_levo["visit_month"] = ledd_levo.VisitTypPDBP.map(lambda x: re.sub(r"months$", "", x))
ledd_levo["visit_month"] = pd.to_numeric(np.where(ledd_levo.visit_month == "Baseline", "0", ledd_levo.visit_month), errors='raise')

Executing: gsutil ls gs://fc-509fb8f7-9df1-43ec-805d-e976c08ce034
gs://fc-509fb8f7-9df1-43ec-805d-e976c08ce034/clinical_data/
gs://fc-509fb8f7-9df1-43ec-805d-e976c08ce034/covar_files/
gs://fc-509fb8f7-9df1-43ec-805d-e976c08ce034/notebooks/


In [48]:
ledd_levo.info()
PDBP_QC = pd.merge(PDBP_QC,
                   ledd_levo[["ID", "visit_month", "LEDD", "DailyLevo"]], 
                   on = ["ID", "visit_month"],
                   #on = "ID",
                   how = "left")

PDBP_QC.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5893 entries, 0 to 5892
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SGUID         5893 non-null   object 
 1   VisitTypPDBP  5893 non-null   object 
 2   LEDD          5893 non-null   float64
 3   DailyLevo     3378 non-null   float64
 4   ID            5893 non-null   object 
 5   visit_month   5893 non-null   int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 276.4+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 330 entries, 0 to 329
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  330 non-null    object 
 1   age_at_baseline     330 non-null    int64  
 2   SEX                 330 non-null    int64  
 3   age_at_diagnosis    298 non-null    float64
 4   AAD_MISSING         330 non-null    int64  
 5   AAD_IMPUTED         330 non-null 

**NOTE TO MYSELF**
Levodop data is unsuitable.  
We are missing data for half of the patients in the QCed data

### PDBP summary statistics
Get metrics for the whole cohort and stratified by event_dyskinesias

In [49]:
PDBP_QC.LONG_DISDUR.unique()

array([0, 1])

In [42]:
PDBP_QC.info()

# Funny casualty is that the numbers below match. I did check this is not coming from an error during processing
print(PDBP_QC["EVENT_DYSK"].value_counts())
print(PDBP_QC["LONG_DISDUR"].value_counts())
# Check the outcome distribution

<class 'pandas.core.frame.DataFrame'>
Int64Index: 330 entries, 0 to 329
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  330 non-null    object 
 1   age_at_baseline     330 non-null    int64  
 2   SEX                 330 non-null    int64  
 3   age_at_diagnosis    298 non-null    float64
 4   AAD_MISSING         330 non-null    int64  
 5   AAD_IMPUTED         330 non-null    float64
 6   DIS_DUR             330 non-null    float64
 7   LONG_DISDUR         330 non-null    int64  
 8   MDSUPDRSIII_BL      330 non-null    float64
 9   visit_month         330 non-null    float64
 10  TIMEMIDPOINT_YEARS  330 non-null    float64
 11  EVENT_DYSK          330 non-null    int64  
 12  LEDD                215 non-null    float64
 13  DailyLevo           181 non-null    float64
dtypes: float64(8), int64(5), object(1)
memory usage: 38.7+ KB
0    286
1     44
Name: EVENT_DYSK, dtype: int64

In [50]:
PDBP_QC.describe(include=np.number)  

,age_at_baseline,SEX,age_at_diagnosis,AAD_MISSING,AAD_IMPUTED,DIS_DUR,LONG_DISDUR,MDSUPDRSIII_BL,visit_month,TIMEMIDPOINT_YEARS,EVENT_DYSK,LEDD,DailyLevo
count,330.000000,330.000000,298.000000,330.000000,330.000000,330.000000,330.000000,330.000000,330.000000,330.000000,330.000000,215.000000,181.000000
mean,63.978788,1.384848,61.536913,0.096970,61.535117,2.924921,0.012121,21.187879,33.290909,2.948485,0.133333,683.476744,552.809392
std,9.328658,0.487298,9.292858,0.296366,8.829369,2.672738,0.109593,11.293589,12.358140,1.071174,0.340451,418.087984,363.586790
min,34.000000,1.000000,33.000000,0.000000,33.000000,0.000000,0.000000,2.000000,6.000000,0.250000,0.000000,60.000000,6.000000
25%,58.000000,1.000000,55.250000,0.000000,56.000000,1.000000,0.000000,13.000000,24.000000,2.250000,0.000000,400.000000,300.000000
50%,65.000000,1.000000,62.000000,0.000000,61.518395,2.000000,0.000000,18.000000,36.000000,3.250000,0.000000,600.000000,450.000000
75%,71.000000,2.000000,68.000000,0.000000,67.000000,5.000000,0.000000,28.750000,42.000000,3.750000,0.000000,893.750000,800.000000
max,87.000000,2.000000,85.000000,1.000000,85.000000,16.481605,1.000000,65.000000,60.000000,5.000000,1.000000,2500.000000,2050.000000


In [44]:
# Checking number of missingness
PDBP_QC.isna().sum()

ID                      0
age_at_baseline         0
SEX                     0
age_at_diagnosis       32
AAD_MISSING             0
AAD_IMPUTED             0
DIS_DUR                 0
LONG_DISDUR             0
MDSUPDRSIII_BL          0
visit_month             0
TIMEMIDPOINT_YEARS      0
EVENT_DYSK              0
LEDD                  115
DailyLevo             149
dtype: int64

#### Observe the sumary statistitc stratified by the disease duration groups

In [51]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(PDBP_QC.groupby('LONG_DISDUR').describe(include=np.number))   

            age_at_baseline                                                 \
                      count       mean       std   min    25%   50%    75%   
LONG_DISDUR                                                                  
0                     326.0  63.855828  9.314046  34.0  58.00  65.0  70.75   
1                       4.0  74.000000  3.162278  71.0  71.75  73.5  75.75   

                     SEX                                               \
              max  count      mean       std  min  25%  50%  75%  max   
LONG_DISDUR                                                             
0            87.0  326.0  1.389571  0.488403  1.0  1.0  1.0  2.0  2.0   
1            78.0    4.0  1.000000  0.000000  1.0  1.0  1.0  1.0  1.0   

            age_at_diagnosis                                                \
                       count       mean       std   min    25%   50%   75%   
LONG_DISDUR                                                                  
0        

### COX PH models fitting
Here, we want to have a look at th models and see whether the COXZP is met and whether the covariates are significantly associated with the time to event outcome

In [52]:
PDBP_QC_SEXAAD = PDBP_QC[["TIMEMIDPOINT_YEARS", "EVENT_DYSK", "AAD_IMPUTED", "SEX"]]


cph = CoxPHFitter()
cph.fit(PDBP_QC_SEXAAD, "TIMEMIDPOINT_YEARS", event_col = "EVENT_DYSK")
cph.print_summary()
cph.check_assumptions(PDBP_QC_SEXAAD, show_plots=True)

<lifelines.CoxPHFitter: fitted with 330 total observations, 286 right-censored observations>
             duration col = 'TIMEMIDPOINT_YEARS'
                event col = 'EVENT_DYSK'
      baseline estimation = breslow
   number of observations = 330
number of events observed = 44
   partial log-likelihood = -242.01
         time fit was run = 2023-04-03 10:13:26 UTC

---
              coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                           
AAD_IMPUTED   0.02       1.02       0.02            -0.02             0.05                 0.98                 1.05
SEX           0.18       1.19       0.30            -0.42             0.77                 0.66                 2.17

              cmp to    z    p   -log2(p)
covariate                                
AAD_IMPUTED     0.00 1.04 0.30       1.74
SEX             0.00 0.58 0.56       0.83
---
Concordance = 0.57
Partial AIC = 488.02
log-likelihood ratio test = 1.46 on 2 df
-log2(p) of ll-ratio test = 1.05

Proportional hazard assumption looks okay.


[]

In [53]:
PDBP_QC_SEXAAD_DISDUR = PDBP_QC[["TIMEMIDPOINT_YEARS", "EVENT_DYSK", "AAD_IMPUTED", "SEX", "DIS_DUR"]]


cph = CoxPHFitter()
cph.fit(PDBP_QC_SEXAAD_DISDUR, "TIMEMIDPOINT_YEARS", event_col = "EVENT_DYSK")
cph.print_summary()

cph.check_assumptions(PDBP_QC_SEXAAD_DISDUR, show_plots=True)

<lifelines.CoxPHFitter: fitted with 330 total observations, 286 right-censored observations>
             duration col = 'TIMEMIDPOINT_YEARS'
                event col = 'EVENT_DYSK'
      baseline estimation = breslow
   number of observations = 330
number of events observed = 44
   partial log-likelihood = -241.97
         time fit was run = 2023-04-03 10:13:28 UTC

---
              coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                           
AAD_IMPUTED   0.02       1.02       0.02            -0.02             0.05                 0.98                 1.06
SEX           0.18       1.20       0.31            -0.42             0.78                 0.66                 2.19
DIS_DUR       0.02       1.02       0.06            -0.10             0.13                 0.91                 1.14

              cmp to    z    p   -log2(p)
covariate                                
AAD_IMPUTED     0.00 1.07 0.28       1.82
SEX             0.00 0.60 0.55       0.87
DIS_DUR         0.00 0.30 0.76       0.39
---
Concordance = 0.57
Partial AIC = 489.93
log-likelihood ratio test = 1.55 on 3 df
-log2(p) of ll-ratio test = 0.57

Proportional hazard assumption looks okay.


[]

In [54]:
PDBP_QC_SEXAAD_DISDUR_MDSUPDRSIII = PDBP_QC[["TIMEMIDPOINT_YEARS", "EVENT_DYSK", "AAD_IMPUTED", "SEX", "DIS_DUR",
                                "MDSUPDRSIII_BL"]]

cph = CoxPHFitter()
cph.fit(PDBP_QC_SEXAAD_DISDUR_MDSUPDRSIII, "TIMEMIDPOINT_YEARS", event_col = "EVENT_DYSK")
cph.print_summary()

cph.check_assumptions(PDBP_QC_SEXAAD_DISDUR_MDSUPDRSIII, show_plots=True)

<lifelines.CoxPHFitter: fitted with 330 total observations, 286 right-censored observations>
             duration col = 'TIMEMIDPOINT_YEARS'
                event col = 'EVENT_DYSK'
      baseline estimation = breslow
   number of observations = 330
number of events observed = 44
   partial log-likelihood = -240.04
         time fit was run = 2023-04-03 10:13:30 UTC

---
                 coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                              
AAD_IMPUTED      0.01       1.01       0.02            -0.02             0.05                 0.98                 1.05
SEX              0.24       1.27       0.31            -0.36             0.85                 0.70                 2.33
DIS_DUR         -0.02       0.98       0.06            -0.15             0.10                 0.86                 1.11
MDSUPDRSIII_BL   0.03       1.03       0.01             0.00             0.05                 1.00                 1.05

                 cmp to     z    p   -log2(p)
covariate                                    
AAD_IMPUTED        0.00  0.60 0.55       0.87
SEX                0.00  0.79 0.43       1.22
DIS_DUR            0.00 -0.34 0.73       0.45
MDSUPDRSIII_BL     0.00  2.03 0.04       4.55
---
Concordance = 0.60
Partial AIC = 488.08
log-likelihood ratio test = 5.40 on 4 df
-log2(p) of ll-ratio test = 2.01

Proportional hazard assumption looks okay.


[]

In [55]:
PDBP_QC_SEXAAD_DISDUR_MDSUPDRSIII_LDOPA = PDBP_QC[["TIMEMIDPOINT_YEARS", "EVENT_DYSK", "AAD_IMPUTED", "SEX", "DIS_DUR",
                                "MDSUPDRSIII_BL", "DailyLevo"]]


PDBP_QC_SEXAAD_DISDUR_MDSUPDRSIII_LDOPA = PDBP_QC_SEXAAD_DISDUR_MDSUPDRSIII_LDOPA[PDBP_QC_SEXAAD_DISDUR_MDSUPDRSIII_LDOPA['DailyLevo'].notnull()]

cph = CoxPHFitter()
cph.fit(PDBP_QC_SEXAAD_DISDUR_MDSUPDRSIII_LDOPA, "TIMEMIDPOINT_YEARS", event_col = "EVENT_DYSK")
cph.print_summary()

cph.check_assumptions(PDBP_QC_SEXAAD_DISDUR_MDSUPDRSIII_LDOPA, show_plots=True)

<lifelines.CoxPHFitter: fitted with 181 total observations, 145 right-censored observations>
             duration col = 'TIMEMIDPOINT_YEARS'
                event col = 'EVENT_DYSK'
      baseline estimation = breslow
   number of observations = 181
number of events observed = 36
   partial log-likelihood = -173.37
         time fit was run = 2023-04-03 10:13:32 UTC

---
                 coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                              
AAD_IMPUTED      0.02       1.02       0.02            -0.02             0.07                 0.98                 1.07
SEX              0.29       1.33       0.35            -0.41             0.98                 0.67                 2.66
DIS_DUR          0.04       1.04       0.07            -0.10             0.18                 0.90                 1.20
MDSUPDRSIII_BL   0.02       1.02       0.01            -0.00             0.05                 1.00                 1.05
DailyLevo       -0.00       1.00       0.00            -0.00             0.00                 1.00                 1.00

                 cmp to     z    p   -log2(p)
covariate                                    
AAD_IMPUTED        0.00  1.07 0.28       1.81
SEX                0.00  0.81 0.42       1.26
DIS_DUR            0.00  0.56 0.58       0.80
MDSUPDRSIII_BL     0.00  1.65 0.10       3.33
DailyLevo          0.00 -0.72 0.47       1.09
---
Concordance = 0.61
Partial AIC = 356.74
log-likelihood ratio test = 6.94 on 5 df
-log2(p) of ll-ratio test = 2.15

Proportional hazard assumption looks okay.


[]

### Sensitivity analysis
The idea is to exclude people with long disease duration and observe the results

In [56]:
PDBP_QC_NODISDUR = PDBP_QC[PDBP_QC.LONG_DISDUR.eq(0)]
PDBP_QC_NODISDUR.info()

print(PDBP_QC_NODISDUR["EVENT_DYSK"].value_counts())
print(PDBP_QC_NODISDUR["LONG_DISDUR"].value_counts())
# Check the outcome distribution

<class 'pandas.core.frame.DataFrame'>
Int64Index: 326 entries, 0 to 329
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ID                  326 non-null    object 
 1   age_at_baseline     326 non-null    int64  
 2   SEX                 326 non-null    int64  
 3   age_at_diagnosis    298 non-null    float64
 4   AAD_MISSING         326 non-null    int64  
 5   AAD_IMPUTED         326 non-null    float64
 6   DIS_DUR             326 non-null    float64
 7   LONG_DISDUR         326 non-null    int64  
 8   MDSUPDRSIII_BL      326 non-null    float64
 9   visit_month         326 non-null    float64
 10  TIMEMIDPOINT_YEARS  326 non-null    float64
 11  EVENT_DYSK          326 non-null    int64  
 12  LEDD                211 non-null    float64
 13  DailyLevo           178 non-null    float64
dtypes: float64(8), int64(5), object(1)
memory usage: 38.2+ KB
0    282
1     44
Name: EVENT_DYSK, dtype: int64

In [57]:
PDBP_QC_SEXAAD_DISDUR_MDSUPDRSIII = PDBP_QC_NODISDUR[["TIMEMIDPOINT_YEARS", "EVENT_DYSK", "AAD_IMPUTED", "SEX", "DIS_DUR",
                                "MDSUPDRSIII_BL"]]

cph = CoxPHFitter()
cph.fit(PDBP_QC_SEXAAD_DISDUR_MDSUPDRSIII, "TIMEMIDPOINT_YEARS", event_col = "EVENT_DYSK")
cph.print_summary()

cph.check_assumptions(PDBP_QC_SEXAAD_DISDUR_MDSUPDRSIII, show_plots=True)

<lifelines.CoxPHFitter: fitted with 326 total observations, 282 right-censored observations>
             duration col = 'TIMEMIDPOINT_YEARS'
                event col = 'EVENT_DYSK'
      baseline estimation = breslow
   number of observations = 326
number of events observed = 44
   partial log-likelihood = -239.79
         time fit was run = 2023-04-03 10:13:37 UTC

---
                 coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                              
AAD_IMPUTED      0.01       1.01       0.02            -0.02             0.05                 0.98                 1.05
SEX              0.24       1.27       0.31            -0.37             0.84                 0.69                 2.31
DIS_DUR         -0.01       0.99       0.07            -0.14             0.12                 0.87                 1.12
MDSUPDRSIII_BL   0.03       1.03       0.01             0.00             0.05                 1.00                 1.05

                 cmp to     z    p   -log2(p)
covariate                                    
AAD_IMPUTED        0.00  0.63 0.53       0.92
SEX                0.00  0.77 0.44       1.18
DIS_DUR            0.00 -0.18 0.86       0.22
MDSUPDRSIII_BL     0.00  2.01 0.04       4.49
---
Concordance = 0.60
Partial AIC = 487.58
log-likelihood ratio test = 5.41 on 4 df
-log2(p) of ll-ratio test = 2.01

Proportional hazard assumption looks okay.


[]

In [58]:
PDBP_QC_SEXAAD_DISDUR_MDSUPDRSIII_LDOPA = PDBP_QC_NODISDUR[["TIMEMIDPOINT_YEARS", "EVENT_DYSK", "AAD_IMPUTED", "SEX", "DIS_DUR",
                                "MDSUPDRSIII_BL", "DailyLevo"]]


PDBP_QC_SEXAAD_DISDUR_MDSUPDRSIII_LDOPA = PDBP_QC_SEXAAD_DISDUR_MDSUPDRSIII_LDOPA[PDBP_QC_SEXAAD_DISDUR_MDSUPDRSIII_LDOPA['DailyLevo'].notnull()]


cph = CoxPHFitter()
cph.fit(PDBP_QC_SEXAAD_DISDUR_MDSUPDRSIII_LDOPA, "TIMEMIDPOINT_YEARS", event_col = "EVENT_DYSK")
cph.print_summary()

cph.check_assumptions(PDBP_QC_SEXAAD_DISDUR_MDSUPDRSIII_LDOPA, show_plots=True)

<lifelines.CoxPHFitter: fitted with 178 total observations, 142 right-censored observations>
             duration col = 'TIMEMIDPOINT_YEARS'
                event col = 'EVENT_DYSK'
      baseline estimation = breslow
   number of observations = 178
number of events observed = 36
   partial log-likelihood = -172.86
         time fit was run = 2023-04-03 10:13:39 UTC

---
                 coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                              
AAD_IMPUTED      0.03       1.03       0.02            -0.02             0.07                 0.98                 1.07
SEX              0.27       1.31       0.35            -0.42             0.96                 0.66                 2.62
DIS_DUR          0.07       1.07       0.08            -0.09             0.22                 0.91                 1.25
MDSUPDRSIII_BL   0.02       1.02       0.01            -0.01             0.05                 0.99                 1.05
DailyLevo       -0.00       1.00       0.00            -0.00             0.00                 1.00                 1.00

                 cmp to     z    p   -log2(p)
covariate                                    
AAD_IMPUTED        0.00  1.13 0.26       1.95
SEX                0.00  0.77 0.44       1.18
DIS_DUR            0.00  0.82 0.41       1.29
MDSUPDRSIII_BL     0.00  1.55 0.12       3.03
DailyLevo          0.00 -0.75 0.46       1.14
---
Concordance = 0.61
Partial AIC = 355.71
log-likelihood ratio test = 7.29 on 5 df
-log2(p) of ll-ratio test = 2.32

Proportional hazard assumption looks okay.


[]

## SAVE PDBPD after QC


In [59]:
shell_do(f'gsutil ls {WORKSPACE_BUCKET}')
ledd_levo = gcs_read_csv(os.path.join(WORKSPACE_BUCKET, 'clinical_data/PDBP_derived_ledd.csv'))

Executing: gsutil ls gs://fc-509fb8f7-9df1-43ec-805d-e976c08ce034
gs://fc-509fb8f7-9df1-43ec-805d-e976c08ce034/clinical_data/
gs://fc-509fb8f7-9df1-43ec-805d-e976c08ce034/covar_files/
gs://fc-509fb8f7-9df1-43ec-805d-e976c08ce034/notebooks/


In [60]:
# Save our merged dataframe out to a .csv and .txt file
csv_output = "PDBP_LID.csv"
txt_output = "PDBP_LID.txt"
PDBP_QC.to_csv(csv_output, index=False, sep=",")
PDBP_QC.to_csv(txt_output, index=False, sep="\t")

In [61]:

shell_do(f'gsutil ls {WORKSPACE_BUCKET}')

Executing: gsutil ls gs://fc-509fb8f7-9df1-43ec-805d-e976c08ce034
gs://fc-509fb8f7-9df1-43ec-805d-e976c08ce034/clinical_data/
gs://fc-509fb8f7-9df1-43ec-805d-e976c08ce034/covar_files/
gs://fc-509fb8f7-9df1-43ec-805d-e976c08ce034/notebooks/


In [62]:
# Save this file to a new directory in your cloud storage 
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {csv_output} {WORKSPACE_BUCKET}/clinical_data/')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {txt_output} {WORKSPACE_BUCKET}/clinical_data/')

Executing: gsutil -u terra-1cae21c2 -m cp PDBP_LID.csv gs://fc-509fb8f7-9df1-43ec-805d-e976c08ce034/clinical_data/
Copying file://PDBP_LID.csv [Content-Type=text/csv]...
/ [1/1 files][ 20.6 KiB/ 20.6 KiB] 100% Done                                    
Operation completed over 1 objects/20.6 KiB.                                     
Executing: gsutil -u terra-1cae21c2 -m cp PDBP_LID.txt gs://fc-509fb8f7-9df1-43ec-805d-e976c08ce034/clinical_data/
Copying file://PDBP_LID.txt [Content-Type=text/plain]...
/ [1/1 files][ 20.6 KiB/ 20.6 KiB] 100% Done                                    
Operation completed over 1 objects/20.6 KiB.                                     
